## アライメントを修正する


Julius によるアラインメントが特定のファイルで大きくズレる問題ですが，Julius の中で無音（振幅ゼロ）の区間を削除してからアラインメントをしていることが原因のようです．削除を行った結果の音声データに対するアラインメント結果をそのまま出力しているので，削除区間の長さの分だけどんどんズレていきます．
自然な録音環境ではそのようなことは起きないという想定なのだと思いますが，J-MAC のデータは後処理か何かで無音区間が結構な割合で入っているデータがあるため問題になるようです．確か松永さんが以前同じ問題に遭遇したと言っていた気がします．おそらく音声ファイルの方で振幅ゼロのサンプルを取り除くとかで対処したのではないかと思いますが，竹下さんの場合は無音区間が削れてしまうのでそれだとダメです．
ログに削除区間の情報が出ているので，それを元にアラインメント結果を修復するプログラムを作りました．添付します．
使い方は以下の通り．
単一の音声ファイルを処理するとき：
``````
# <log-file> : Julius のアラインメントツールから出る *.log ファイル
# <wav-file> : 元の音声ファイル
# <lab-file> : Julius のアラインメントツールから出る *.lab ファイル
# <out-file> : 出力ファイル（lab 形式）
``````
``````
$ fix_align.py <log-file> <wav-file> <lab-file> <out-file>
```
あるディレクトリ以下の *.log, *.wav, *.lab の3つ組み全てについて修正結果を *.lab2 というファイルに入れる：
``````
$ fix_align.py -d directory
``````
ログファイルに Warning: strip: sample から始まる行が出ているファイルは上記の問題が起きています．
問題が起きてないファイルを処理したときはもとのと同じ結果が出るはずなので，全部 fix_align.py で処理しても大丈夫なはずです（エラーが起きなければ）．
竹下さんの結果のディレクトリを見ると，また巨大なログファイルがいくつかできているようです．
それは削除してから実行してください（でないとすごい時間がかかると思います）
あと細かい点ですが，プログラムの中の restore_silence という関数で threshold という引数があり，デフォルトで 1600 サンプル（= 100ms）を指定してあります．これ以上の長さの無音区間は新しく音素 sp の区間を作るようになっています．長さがそれ未満の無音区間は対応する音素の区間に無音区間の長さを加えています．竹下さんのポーズ検出の都合に合わせて設定してください．元ファイルで促音のところが無音になっており，上記の処理の結果 sp が入ってしまうケースもあります．

In [ ]:
import csv
import yaml
import os
import glob
import sys

sys.path.append("/home/takeshun256/PausePrediction/src/analyze_jmac")
sys.path.append("/home/takeshun256/PausePrediction/src/vad_tool")
from audiobook_yaml_parser import extract_yaml_data
from py_webrtcvad_test import getVadSection
from vad_tool import VAD_Segmenter
from audiobook_dataset_builder import AudiobookDatasetBuilder
from audiobook_script_extractor import AudiobookScriptExtractor

# python==3.11以上の場合はimportしない
if sys.version_info < (3, 11):
    from text_preprocessing_preprocessor import AudiobookScriptPreprocessor
# from text_preprocessing import AudiobookScriptPreprocessor
from julius_lab_analysis import JuliusLabAnalyzer

import numpy as np
import scipy
import scipy.io
from scipy.io import wavfile
import scipy.io.wavfile
import scipy.ndimage
import scipy.signal

%matplotlib inline
from matplotlib import pyplot as plt
import japanize_matplotlib
import webrtcvad
from pprint import pprint
import pandas as pd
import seaborn as sns
import librosa
import struct
import librosa.display
from IPython.display import Audio
from tqdm import tqdm
from typing import List, Tuple, Dict, Any, Union
import soundfile as sf
from pathlib import Path

In [ ]:
LAB_DIR = "/data2/takeshun256/jmac_split_and_added_lab"
assert os.path.exists(LAB_DIR)

# ディレクトリ内のlabファイルを取得
lab_files = list(Path(LAB_DIR).glob("**/*.lab"))

print("labファイル数:", len(list(lab_files)))
print("labファイル例:", list(lab_files)[0])

# labファイルを読み込み
lab_analyzer = JuliusLabAnalyzer(lab_files)
# df_lab = lab_analyzer.load_lab_files()
lab_analyzer.set_save_dir("/home/takeshun256/PausePrediction/data_pub/jmac")
# lab_analyzer.save_df_lab_to_pickle(df_lab_attached)
df_lab_attached = lab_analyzer.load_df_lab_from_pickle()
df_lab_attached = df_lab_attached[
    [
        "start",
        "end",
        "phoneme",
        "phoneme_idx",
        "duration",
        "lab_filepath",
        "audiobook_id",
        "audiobook_id_int",
        "chapter_id",
        "lab_idx",
        "chapter_id_int",
        "author",
        "book",
        "text",
        "character",
        "to_whom",
    ]
]

# wavファイルのパスを取得
df_lab_attached["wav_filepath"] = df_lab_attached["lab_filepath"].apply(
    lambda x: str(x).replace(".lab", ".wav")
)

df_lab_attached.head()

## サンプルを1つ選択して修正実行

In [ ]:
# 音声波形抽出
# wav_path -> wav, sr
def extract_waveform(audio_file_path, sr=24000):
    waveform, sample_rate = librosa.load(audio_file_path, mono=True)
    return waveform, sample_rate


# wav -> db変換
def convert_db(waveform):
    # TODO: これは何が違うのか？どちらが適切か？
    # db = librosa.power_to_db(waveform)
    db = librosa.amplitude_to_db(waveform)
    return db


# 連続区間抽出
# db -> bool_list
def run_length_encoding(arr, min_run_length=3):
    diff = np.diff(arr)  # 隣接要素の差分を計算
    run_starts = np.where(diff != 0)[0] + 1  # 差分が0でないインデックスを取得し、連続する範囲の開始位置を得る
    run_lengths = np.diff(np.concatenate(([0], run_starts, [len(arr)])))  # 連続する範囲の長さを計算
    result = np.repeat(run_lengths >= min_run_length, run_lengths)  # 連続する範囲をTrueに変換
    return result


# Pause区間抽出
# db, db_threshold, time_threshold, sr -> pause_bool_list
# 閾値を超えたらpauseとみなす
def detect_pause_position(
    db_sequence, db_threshold=-50, time_threshold=50 / 1000, sample_rate=24000
):
    """dbと音声長の閾値からpauseの位置を判定する。

    Args:
        db_sequence (np.array): 音声波形をdbに変換した配列
        db_threshold (float): 無音区間とするdbの閾値
        time_threshold (float): 無音区間が連続した時にpauseとみなす時間の閾値

    Returns:
        pause_positions (list): pauseの位置のリスト
    """
    under_db_threshold = db_sequence < db_threshold

    # 連続区間を抽出
    sample_threshold = int(time_threshold * sample_rate)
    is_continuous = run_length_encoding(under_db_threshold, sample_threshold)

    # pauseの位置を抽出
    pause_positions = under_db_threshold & is_continuous

    return pause_positions


# pause区間付きの波形の可視化
def plot_db_with_pause(db, sr, db_threshold, time_threshold, xlim=None):
    fig, ax = plt.subplots(figsize=(20, 5))
    x = np.arange(len(db)) / sr
    ax.plot(x, db, label="db")

    # dbの閾値を引く
    ax.axhline(
        y=db_threshold,
        color="r",
        linestyle="-",
        linewidth=2,
        alpha=0.7,
        label="db_threshold",
    )

    # pauseの領域を塗りつぶす
    pause_position = detect_pause_position(db, db_threshold, time_threshold, sr)
    plt.fill_between(x, -80, 0, where=pause_position, facecolor="b", alpha=0.5)

    ax.set_xlim(xlim)
    ax.legend()
    plt.show()


# 波形の可視化
def plot_wavform(waveform, sr, xlim=None):
    fig, ax = plt.subplots(figsize=(20, 5))
    x = np.arange(len(waveform)) / sr
    ax.plot(x, waveform, label="waveform")
    ax.set_xlim(xlim)
    ax.legend()
    plt.show()


# 音声再生ボタン生成
def play_button(waveform, sr):
    display(Audio(waveform, rate=sr, autoplay=True))


# アライメントの抽出
# lab_path -> df_lab
def read_lab(lab_path):
    """labファイルを読み込む"""
    # labファイルがない場合
    if not Path(lab_path).exists():
        print(f"{lab_path} does not exist.")
        return None

    # labファイルがある場合
    df_lab = []
    with open(lab_path, "r") as f:
        for phoneme_idx, line in enumerate(f):
            if line == "":
                continue
            start, end, phoneme = line.split()
            duration = float(end) - float(start)
            df_lab.append(
                {
                    "start": float(start),
                    "end": float(end),
                    "phoneme": phoneme,
                    "phoneme_idx": phoneme_idx,
                    "duration": duration,
                }
            )
    df_lab = pd.DataFrame(df_lab)
    return df_lab


# アライメントの可視化
def plot_phoneme_alignment(df, xlim=None):
    """Labファイルから音素のアライメントをプロットする

    Args:
        lab_path (_type_): Labファイルのパス
    """
    # df = read_lab(lab_path)
    # display(df[-10:])

    # 描画
    fig, ax = plt.subplots(figsize=(20, 2))
    for start, end, label in df.values:
        ax.axvline(start, color="gray", linestyle="--")
        ax.text((start + end) / 2, 0.5, label, ha="center", va="bottom", fontsize=8)
    # ax.set_yticks([])
    ax.set_xlim(xlim)
    ax.set_xlabel("Time (seconds)")
    fig.tight_layout()
    plt.legend()
    plt.show()


# 並べて可視化する。
def plot_all(
    df_temp, wav_path, sample_rate=24000, db_threshold=-50, time_threshold=50 / 1000
):
    """wavファイル、db、アライメントを並べて可視化する"""
    wav, sr = extract_waveform(wav_path, sr=sample_rate)
    db = convert_db(wav)
    xlim = (0, len(wav) / sr)

    print(wav_path)
    print("wav.shape:", wav.shape)
    print("seconds:", len(wav) / sr)

    fig, ax = plt.subplots(
        3, 1, figsize=(20, 10), gridspec_kw={"height_ratios": [4, 4, 2]}
    )
    # print("spk_id:", spk_id)
    # print("wav_id:", wav_id)
    # print("xlim:", xlim)
    # print("transcript:", transcript)
    # print("start ploting...")

    # 波形の可視化
    x = np.arange(len(wav)) / sr
    ax[0].plot(x, wav, label="waveform")
    ax[0].set_xlim(xlim)
    ax[0].legend()

    # dbの可視化
    x = np.arange(len(db)) / sr
    ax[1].plot(x, db, label="db")
    # dbの閾値を引く
    ax[1].axhline(
        y=db_threshold,
        color="r",
        linestyle="-",
        linewidth=2,
        alpha=0.7,
        label="db_threshold",
    )
    # pauseの領域を塗りつぶす
    pause_position = detect_pause_position(db, db_threshold, time_threshold, sr)
    ax[1].fill_between(x, -80, 0, where=pause_position, facecolor="b", alpha=0.5)
    ax[1].set_xlim(xlim)
    ax[1].legend()

    # アライメントの可視化
    # 描画
    for start, end, label in df_temp.values:
        ax[2].axvline(end, color="gray", linestyle="--")
        ax[2].axvline(start, color="gray", linestyle="--")
        ax[2].text((start + end) / 2, 0.5, label, ha="center", va="bottom", fontsize=8)
    # ax.set_yticks([])
    ax[2].set_xlim(xlim)
    ax[2].set_xlabel("Time (seconds)")
    # ax[2].tight_layout()
    ax[2].legend()

    plt.show()

    play_button(wav, sr)


# 並べて可視化する。
def plot_all2(
    df_temp, wav_path, sample_rate=24000, db_threshold=-50, time_threshold=50 / 1000
):
    """wavファイル、db、アライメントを並べて可視化する"""
    wav, sr = extract_waveform(wav_path, sr=sample_rate)
    db = convert_db(wav)
    xlim = (0, len(wav) / sr)

    print(wav_path)
    print("wav.shape:", wav.shape)
    print("seconds:", len(wav) / sr)

    fig, ax = plt.subplots(
        2, 1, figsize=(20, 10), gridspec_kw={"height_ratios": [4, 4]}
    )
    # print("spk_id:", spk_id)
    # print("wav_id:", wav_id)
    # print("xlim:", xlim)
    # print("transcript:", transcript)
    # print("start ploting...")

    # 波形の可視化
    x = np.arange(len(wav)) / sr
    ax[0].plot(x, wav, label="waveform")
    ax[0].set_xlim(xlim)
    ax[0].legend()

    # dbの可視化
    x = np.arange(len(db)) / sr
    ax[1].plot(x, db, label="db")
    # dbの閾値を引く
    ax[1].axhline(
        y=db_threshold,
        color="r",
        linestyle="-",
        linewidth=2,
        alpha=0.7,
        label="db_threshold",
    )
    # pauseの領域を塗りつぶす
    pause_position = detect_pause_position(db, db_threshold, time_threshold, sr)
    ax[1].fill_between(x, -80, 0, where=pause_position, facecolor="b", alpha=0.5)
    ax[1].set_xlim(xlim)
    ax[1].legend()

    # アライメントの可視化
    # 描画
    for start, end, label in df_temp.values:
        ax[0].axvline(end, color="gray", linestyle="--")
        ax[0].axvline(start, color="gray", linestyle="--")
        ax[0].text((start + end) / 2, 0.5, label, ha="center", va="bottom", fontsize=8)
    # ax.set_yticks([])
    ax[0].set_xlim(xlim)
    ax[0].set_xlabel("Time (seconds)")
    # ax[2].tight_layout()
    ax[0].legend()

    plt.show()

    play_button(wav, sr)


def classfy_pause(
    db_sequence, lab_path, sample_rate=24000, db_threshold=-50, time_threshold=0.05
):
    """ポーズを分類する

    Args:
        df_jvs (_type_): _description_
    """
    # db_threshold = -50
    # time_threshold = 0.05
    # sample_rate = 24000

    # db_sequence = df_jvs.iloc[0]['db_sequence']
    pause_position = detect_pause_position(
        db_sequence, db_threshold, time_threshold, sample_rate
    )

    def run_length_encoding_range(arr, min_run_length=3):
        """
        Run-Length Encoding (RLE)を実行して連続している部分をTrueとしたブール配列を返す関数

        Parameters:
            arr (numpy.ndarray): 連続している部分を判定したい1次元のNumPy配列
            min_run_length (int): 連続していると判定する最小の長さ（デフォルトは3）

        Returns:
            numpy.ndarray: 連続している部分がTrueとなったブール配列
            list: 連続している部分の始点と終点のリスト [(start1, end1), (start2, end2), ...]
        """
        diff = np.diff(arr)  # 隣接要素の差分を計算
        run_starts = np.where(diff != 0)[0] + 1  # 差分が0でないインデックスを取得し、連続する範囲の開始位置を得る

        starts = np.concatenate(([0], run_starts))
        ends = np.concatenate((run_starts, [len(arr)]))
        lengths = ends - starts
        ranges = list(zip(starts, ends, lengths))

        # min_run_length以下の範囲を削除, Trueが連続しているもののみを取り出す
        ranges = [r for r in ranges if (r[2] >= min_run_length and arr[r[0]])]

        return ranges

    sample_threshold = int(time_threshold * sample_rate)
    pause_ranges = run_length_encoding_range(pause_position, sample_threshold)

    # print(pause_ranges)

    # df_lab = read_lab(df_jvs.iloc[0]['lab_path'])
    df_lab = read_lab(lab_path)

    ans = []
    for pause_range in pause_ranges:
        # df_labのstartもしくは、endが、start, endの範囲内にあるかどうか
        pause_start = pause_range[0]
        pause_end = pause_range[1]
        phoneme_start = df_lab["start"].values * sample_rate
        phoneme_end = df_lab["end"].values * sample_rate
        is_start_include = (pause_start <= phoneme_start) & (phoneme_start <= pause_end)
        is_end_include = (pause_start <= phoneme_end) & (phoneme_end <= pause_end)

        include_phonemes = df_lab[is_start_include | is_end_include]["phoneme"].values
        print(include_phonemes)
        if "silE" in include_phonemes:
            pause_type = "silE"
        elif "silB" in include_phonemes:
            pause_type = "silB"
        elif "sil" in include_phonemes:
            pause_type = "sil"
        elif "pau" in include_phonemes:
            pause_type = "pau"
        elif "sp" in include_phonemes:
            pause_type = "sp"
        else:
            pause_type = "RP"

        ans.append([pause_range[0], pause_range[1], pause_range[2], pause_type])
    return ans

In [ ]:
# もう1つのwavファイル
# 波形の上にpauseの位置を可視化する

# ----setting----
# 閾値の設定
db_threshold = -50
# time_threshold = 50 / 1000 # 50ms
time_threshold = 200 / 1000  # 200ms
sample_rate = 24000
# sample_rate = 16000
# ---------------

wav_filepath = df_lab_attached.iloc[999]["wav_filepath"]
wav_filepath = (
    "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.wav"
)

df_temp = df_lab_attached[df_lab_attached["wav_filepath"] == wav_filepath][
    ["start", "end", "phoneme"]
]
print(df_temp)
print(df_lab_attached[df_lab_attached["wav_filepath"] == wav_filepath]["text"])

plot_all2(
    df_temp,
    wav_filepath,
    sample_rate=sample_rate,
    db_threshold=db_threshold,
    time_threshold=time_threshold,
)

### サンプル：/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.wav に修正を実行

In [ ]:
!cat "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.txt"

In [ ]:
!cat "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.lab"

In [ ]:
!cat "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.log"

In [ ]:
!cat "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.log" | grep "Warning"

In [ ]:
# !/home/takeshun256/PausePrediction/src/julius_segment/fix_align.py "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.log" "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.wav" "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.lab" "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.lab2"

# !./fix_align_all.sh > "/home/takeshun256/PausePrediction/logs/fix_align_all_2023-11-01_11:21.log" 2>&1

In [ ]:
!cat "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.lab2"

### 修正後の結果を確認

In [ ]:
# もう1つのwavファイル
# 波形の上にpauseの位置を可視化する

# ----setting----
# 閾値の設定
db_threshold = -50
# time_threshold = 50 / 1000 # 50ms
time_threshold = 200 / 1000  # 200ms
sample_rate = 24000
# sample_rate = 16000
# ---------------

wav_filepath = (
    "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.wav"
)
lab_filepath = (
    "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.lab2"
)
txt_filepath = (
    "/data2/takeshun256/jmac_split_and_added_lab/audiobook_72/audiobook_72_079.txt"
)


df_temp = read_lab(lab_filepath)[["start", "end", "phoneme"]]
print(df_temp)
with open(txt_filepath, "r") as f:
    transcript = f.read()
print(transcript)

plot_all2(
    df_temp,
    wav_filepath,
    sample_rate=sample_rate,
    db_threshold=db_threshold,
    time_threshold=time_threshold,
)

## 修正中にエラーが出ているものを確認

- 確認方法
    - cat "/data2/takeshun256/jmac_split_and_added_lab/audiobook_33/audiobook_33_110.log" | grep War
- 結果
    - Warning: strip: sample 77378-77398 has zero value, stripped

In [ ]:
# もう1つのwavファイル
# 波形の上にpauseの位置を可視化する

# ----setting----
# 閾値の設定
db_threshold = -50
# time_threshold = 50 / 1000 # 50ms
time_threshold = 200 / 1000  # 200ms
sample_rate = 24000
# sample_rate = 16000
# ---------------

audiobook_id = 33
sample_id = 110

wav_filepath = f"/data2/takeshun256/jmac_split_and_added_lab/audiobook_{audiobook_id}/audiobook_{audiobook_id}_{sample_id:03}.wav"
lab_filepath = f"/data2/takeshun256/jmac_split_and_added_lab/audiobook_{audiobook_id}/audiobook_{audiobook_id}_{sample_id:03}.lab"
txt_filepath = f"/data2/takeshun256/jmac_split_and_added_lab/audiobook_{audiobook_id}/audiobook_{audiobook_id}_{sample_id:03}.txt"


df_temp = read_lab(lab_filepath)[["start", "end", "phoneme"]]
print(df_temp)
with open(txt_filepath, "r") as f:
    transcript = f.read()
print(transcript)

plot_all2(
    df_temp,
    wav_filepath,
    sample_rate=sample_rate,
    db_threshold=db_threshold,
    time_threshold=time_threshold,
)